In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse as sp
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
# from sklearn.neighbors.nearest_centroid import NearestCentroid
import time
from sys import getsizeof

In [4]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.validation import check_array, check_X_y, check_is_fitted
from sklearn.utils.sparsefuncs import csc_median_axis_0
from sklearn.utils.multiclass import check_classification_targets

# This is a modified version of NearestCentroid from sklearn lib
class NearestCentroid(BaseEstimator, ClassifierMixin):

    def __init__(self, metric='euclidean', shrink_threshold=None):
        self.metric = metric
        self.shrink_threshold = shrink_threshold

    def fit(self, X, y):

        if self.metric == 'precomputed':
            raise ValueError("Precomputed is not supported.")
        # If X is sparse and the metric is "manhattan", store it in a csc
        # format is easier to calculate the median.
        if self.metric == 'manhattan':
            X, y = check_X_y(X, y, ['csc'])
        else:
            X, y = check_X_y(X, y, ['csr', 'csc'])
        is_X_sparse = sp.issparse(X)
        if is_X_sparse and self.shrink_threshold:
            raise ValueError("threshold shrinking not supported"
                             " for sparse input")
        check_classification_targets(y)

        n_samples, n_features = X.shape
        le = LabelEncoder()
        y_ind = le.fit_transform(y)
        self.classes_ = classes = le.classes_
        n_classes = classes.size
        if n_classes < 2:
            raise ValueError('The number of classes has to be greater than'
                             ' one; got %d class' % (n_classes))

        # Mask mapping each class to its members.
        self.centroids_ = sp.lil_matrix((n_classes, n_features), dtype=np.float64)
        # Number of clusters in each class.
        nk = np.zeros(n_classes)

        for cur_class in range(n_classes):
            center_mask = y_ind == cur_class
            nk[cur_class] = np.sum(center_mask)
            if is_X_sparse:
                center_mask = np.where(center_mask)[0]

            # XXX: Update other averaging methods according to the metrics.
            if self.metric == "manhattan":
                # NumPy does not calculate median of sparse matrices.
                if not is_X_sparse:
                    self.centroids_[cur_class] = np.median(X[center_mask], axis=0)
                else:
                    self.centroids_[cur_class] = csc_median_axis_0(X[center_mask])
            else:
                if self.metric != 'euclidean':
                    warnings.warn("Averaging for metrics other than "
                                  "euclidean and manhattan not supported. "
                                  "The average is set to be the mean."
                                  )
                self.centroids_[cur_class] = X[center_mask].mean(axis=0)

        if self.shrink_threshold:
            dataset_centroid_ = np.mean(X, axis=0)

            # m parameter for determining deviation
            m = np.sqrt((1. / nk) - (1. / n_samples))
            # Calculate deviation using the standard deviation of centroids.
            variance = (X - self.centroids_[y_ind]) ** 2
            variance = variance.sum(axis=0)
            s = np.sqrt(variance / (n_samples - n_classes))
            s += np.median(s)  # To deter outliers from affecting the results.
            mm = m.reshape(len(m), 1)  # Reshape to allow broadcasting.
            ms = mm * s
            deviation = ((self.centroids_ - dataset_centroid_) / ms)
            # Soft thresholding: if the deviation crosses 0 during shrinking,
            # it becomes zero.
            signs = np.sign(deviation)
            deviation = (np.abs(deviation) - self.shrink_threshold)
            np.clip(deviation, 0, None, out=deviation)
            deviation *= signs
            # Now adjust the centroids using the deviation
            msd = ms * deviation
            self.centroids_ = dataset_centroid_[np.newaxis, :] + msd
        return self

    def predict(self, X):

        check_is_fitted(self, 'centroids_')

        X = check_array(X, accept_sparse='csr')
        
        return np.argsort(pairwise_distances(X, self.centroids_, metric=self.metric))[:10]


In [3]:
y_doc_vect_representation = pickle.load(open('y_doc_vect_representation', 'rb'))
X_doc_vect_representation = pickle.load(open('X_doc_vect_representation', 'rb'))

In [5]:
# print(y_doc_vect_representation[1][1])

clf = NearestCentroid()
clf.fit(X_doc_vect_representation, y_doc_vect_representation[0])

True


In [7]:
pickle.dump(clf, open("clf_all_data1", "wb"))

In [3]:
vectorizer = pickle.load(open('vectorizer', 'rb'))

In [6]:
title = ["Filogenia molecular e análise morfológica de espécies de Paralucilia Brauer & Bergenstamm, 1891 (Insecta, Diptera, Calliphoridae)",
        "Um método para deduplicação de metadados bibliográficos baseado no empilhamento de classificadores",
        "Modelo RHA - Retroalimentação em Hipermídia Adaptativa"]
abstract = ["Paralucilia Brauer & Bergenstamm, 1891 (Insecta, Diptera, Calliphoridae) é um gênero que compreende espécies Neotropicais comumente observadas associadas a cadáveres e carcaças em ambientes sem, ou com pouca perturbação antrópica. Divergências na literatura quanto ao número e nomenclatura das espécies válidas levam à falta de consenso na denominação das espécies em estudos que levantam informações sobre a biodiversidade e os de importância forense. Neste estudo foi proposta a primeira hipótese filogenética baseada em dados moleculares (dos genes COI, ITS2, 16S e 28S) para diferentes morfotipos de Paralucilia [P. fulvinota, P. pseudolyrcea, P. paraensis, Paralucilia sp.1 (=P. nigrofacialis), Paralucilia sp.2 e Paralucilia sp.3 (=P. xanthogeneiates)], visando resolver os conflitos taxonômicos do gênero. Análises de Máxima Parcimônia, Máxima Verossimilhança e Inferência Bayesiana recuperaram o grupo como sendo monofilético, tal como tem sido observado para os demais gêneros de Chrysomyinae Neotropicais. Após todas as análises concluídas destacamos que devem ser consideradas como espécies válidas para o gênero: P. fulvinota, P. pseudolyrcea e P. paraensis. Propomos a sinonimização de Paralucilia nigrofacialis para P. fulvinota, uma vez que se refere a uma variação morfológica polimórfica desta espécie; e a sinonimização de P. xanthogeneiates para P. pseudolyrcea. O status de P. borgmeieri deve ser mantido como incerto até que estudos mais aprofundados possam ser conduzidos, além da necessidade de designação de um lectótipo. A partir das análises morfológicas foram considerados os seguintes caracteres como sendo de valor diagnóstico para as espécies de Paralucilia: cor da microtricosidade da pós-gena; cor dos cílios da pós-gena; número de faixas longitudinais no tórax; cor do espiráculo anterior; e número de cerdas posterodorsais da tíbia posterior. Além disso, o polimorfismo de P. fulvinota foi descrito mais aprofundadamente visando evitar o surgimento de novos conflitos taxonômicos. Este estudo contribuiu para o esclarecimento da taxonomia de Paralucilia, e as informações aqui levantadas vêm a ser úteis a fim de minimizar o impedimento taxonômico entre os dípteros Neotropicais.",
            "Metadados bibliográficos duplicados são registros que correspondem a referências bibliográficas semanticamente equivalentes, ou seja, que descrevem a mesma publicação. Identificar metadados bibliográficos duplicados em uma ou mais bibliotecas digitais é uma tarefa essencial para garantir a qualidade de alguns serviços como busca, navegação e recomendação de conteúdo. Embora diversos padrões de metadados tenham sido propostos, eles não resolvem totalmente os problemas de interoperabilidade porque mesmo que exista um mapeamento entre diferentes esquemas de metadados, podem existir variações na representação do conteúdo. Grande parte dos trabalhos propostos para identificar duplicatas aplica uma ou mais funções sobre o conteúdo de determinados campos no intuito de captar a similaridade entre os registros. Entretanto, é necessário escolher um limiar que defina se dois registros são suficientemente similares para serem considerados semanticamente equivalentes ou duplicados. Trabalhos mais recentes tratam a deduplicação de registros como um problema de classificação de dados, em que um modelo preditivo é treinado para estimar a que objeto do mundo real um registro faz referência. O objetivo principal desta tese é o desenvolvimento de um método efetivo e automático para identificar metadados bibliográficos duplicados, combinando o aprendizado de múltiplos classificadores supervisionados, sem a necessidade de intervenção humana na definição de limiares de similaridade. Sobre o conjunto de treinamento são aplicadas funções de similaridade desenvolvidas especificamente para o contexto de bibliotecas digitais e com baixo custo computacional. Os escores produzidos pelas funções são utilizados para treinar múltiplos modelos de classificação heterogêneos, ou seja, a partir de algoritmos de diversos tipos: baseados em árvores, regras, redes neurais artificiais e probabilísticos. Os classificadores aprendidos são combinados através da estratégia de empilhamento visando potencializar o resultado da deduplicação a partir do conhecimento heterogêneo adquirido individualmente pelos algoritmo de aprendizagem. O modelo de classificação final é aplicado aos pares candidatos ao casamento retornados por uma estratégia de blocagem de dois níveis bastante eficiente. A solução proposta é baseada na hipótese de que o empilhamento de classificadores supervisionados pode aumentar a qualidade da deduplicação quando comparado a outras estratégias de combinação. A avaliação experimental mostra que a hipótese foi confirmada quando o método proposto é comparado com a escolha do melhor classificador e com o voto da maioria. Ainda são analisados o impacto da diversidade dos classificadores no resultado do empilhamento e os casos de falha do método proposto.",
            "A presente pesquisa tem como objetivo propor uma extensão para modelos de referência em hipermídia adaptativa de acordo com resultados de avaliação de aprendizagem e estilos cognitivos dos aprendizes. É proposto, portanto, um modelo denominado RHA, Modelo de Retroalimentação em Hipermídia Adaptativa, que utiliza os resultados obtidos com as avaliações de aprendizagem e as definições derivadas dos estilos cognitivos dos aprendizes a fim de prover a retroalimentação dos demais modelos existentes em um hipermídia adaptativo. Na concepção do RHA foi adotada a representação por meio da UML (Unified Modeling Language) com objetivo de diminuir o risco de ambigüidade, facilitar o processo de modelagem computacional, proporcionar o reaproveitamento do modelo e otimizar a implementação do mesmo. Para apoiar a criação do RHA, foi adotado o modelo de referência Munich. Este utiliza UML e apresenta uma arquitetura que contempla módulos tradicionais dos hipermídias adaptativos, tais como: modelo de domínio, de usuário e de adaptatividade; porém, como os demais modelos de referência atuais, este não contempla a definição de itens explícitos à reutilização dos dados obtidos com as avaliações de aprendizagem. O modelo RHA foi criado como uma extensão do Munich, com concepção fundamentada em duas dimensões de estilos cognitivos estabelecidos (MESSICK, 1976), que nortearam a escolha de instrumentos de avaliação de aprendizagem destinados à modalidade ensino a distância. Os instrumentos de avaliação de aprendizagem abrangem atividades definidas de acordo com um grupo de ferramentas de comunicação e interação (síncronas e assíncronas) amplamente adotadas no ensino a distância. A modelagem do RHA envolve aspectos relativos a UML, como a criação das classes com seus atributos e métodos e os relacionamentos entre as classes já existentes no Munich. Para simular a aplicação do modelo RHA, foi definido um domínio de conhecimento relacionado à área de apoio ao ensino sobre Mercado de Capitais. Este tema se mostrou adequado, pois dada à quantidade de materiais e informações disponíveis sobre o assunto, é relevante a adoção de diferentes estilos de aprendizagem com tipos particulares de conteúdos e empregos das ferramentas de comunicação e interação, visando à avaliação de aprendizagem, para públicos distintos. Atualmente o número de cursos a distância no referido domínio de conhecimento é escasso, o que o torna ainda mais relevante para exploração."]
y = [[546437, 582037],[571050],[574936,556151]]

for iii in range(0,3):
    X_test = vectorizer.transform([title[iii] + " " + abstract[iii]])
    y_test = y[iii]
    for ii in range(1,11):
        clf = pickle.load(open('clfs/fold'+str(ii)+'_clf', 'rb'))
        out = clf.predict(X_test)
        print('Ranking fold '+str(ii)+' document '+str(iii)+':')
        for i in range(0,70000):
            if clf.classes_[out[0][i]] in y_test:
                print(str(i)+': '+str(clf.classes_[out[0][i]])+' <---')
            elif i in [0,1,2,3]:
                print(str(i)+': '+str(clf.classes_[out[0][i]]))

Ranking fold 1 document 0:
0: 551350
1: 570726
2: 538029
3: 559202
15829: 546437 <---
65633: 582037 <---
Ranking fold 2 document 0:
0: 551350
1: 516697
2: 570726
3: 538029
15876: 546437 <---
65813: 582037 <---
Ranking fold 3 document 0:
0: 516697
1: 570726
2: 551350
3: 540612
15882: 546437 <---
65641: 582037 <---
Ranking fold 4 document 0:
0: 570726
1: 516697
2: 551350
3: 538029
15852: 546437 <---
65077: 582037 <---
Ranking fold 5 document 0:
0: 551350
1: 516697
2: 538029
3: 540612
15852: 546437 <---
66405: 582037 <---
Ranking fold 6 document 0:
0: 570726
1: 516697
2: 551350
3: 538029
15891: 546437 <---
64857: 582037 <---
Ranking fold 7 document 0:
0: 551350
1: 516697
2: 570726
3: 538029
15874: 546437 <---
69387: 582037 <---
Ranking fold 8 document 0:
0: 570726
1: 516697
2: 551350
3: 578980
24169: 546437 <---
65516: 582037 <---
Ranking fold 9 document 0:
0: 570726
1: 551350
2: 516697
3: 538029
27247: 546437 <---
65530: 582037 <---
Ranking fold 10 document 0:
0: 551350
1: 516697
2: 5707